# Claims

## Set Up

In [1]:
import pandas as pd
import numpy as np
import json
import pickle

## Load metadata

In [2]:
train_set = pd.read_pickle('train_set.pkl')

In [3]:
train_set.head(10)

,claim,claimant,date,label,related_articles,id
9389,While arguing over President Reagan’s 1981 tax...,Sarah Sanders,2017-10-31,1,"[34218, 55700, 18736, 39031, 34219, 34220]",10354
1861,"Recently Rick Scott ""closed 30 women’s health ...",Lois Frankel,2014-09-12,0,"[73190, 76997, 38841, 77415, 77303, 9280, 8332...",2053
11035,Says Target installed urinals in a women’s bat...,Facebook posts,2016-04-22,0,"[9619, 22197]",12160
12221,"Says ""combined doses of vaccines"" have never b...",Facebook posts,2019-04-15,0,"[57163, 31528, 40908, 31536, 68904, 44601]",13458
11354,: The AMBER Alert system has been discontinu...,,2013-10-13,0,"[103978, 121475, 121849]",12504
8662,"Judge Gonzalo Curiel ""is giving us very unfair...",Donald Trump,2016-06-05,0,"[77208, 21821, 63461, 58883]",9554
14307,Continental Airlines CEO Gordon Bethune once p...,,2016-10-23,0,"[112600, 114369, 114400, 123364]",15756
31,"""Expanding Medicaid would require borrowing mo...",Will Weatherford,2013-05-09,1,"[66749, 1228, 7897, 10786]",36
12687,Says a Washington Post reporter who broke news...,Bloggers,2017-11-10,0,"[12178, 53283, 89573, 19796]",13981
6422,About half the people who would qualify for Me...,Will Weatherford,2014-02-27,1,"[93033, 9294, 7826, 11450, 93049, 6737, 92851,...",7062


In [4]:
len(train_set)

12444

In [5]:
test_set = pd.read_pickle('test_set.pkl')

In [6]:
len(test_set)

3111

## Claimaints

In [7]:
claimaints = pd.DataFrame(train_set, columns=['claim', 'claimant', 'label'])
claimaints

,claim,claimant,label
9389,While arguing over President Reagan’s 1981 tax...,Sarah Sanders,1
1861,"Recently Rick Scott ""closed 30 women’s health ...",Lois Frankel,0
11035,Says Target installed urinals in a women’s bat...,Facebook posts,0
12221,"Says ""combined doses of vaccines"" have never b...",Facebook posts,0
11354,: The AMBER Alert system has been discontinu...,,0
...,...,...,...
2910,Health insurance costs for Floridians are up 3...,Republican Party of Florida,1
6096,"A photograph captures Harriet Tubman as a ""Gun...",,0
10446,"ISIS leader Abu Bakr al Baghdadi was ""released...",Jeanine Pirro,0
5414,"""The board of a nonprofit organization on whic...",Tennessee Republican Party,1


In [8]:
claimaints['claimant'].replace('', np.nan, inplace=True)
claimaints

,claim,claimant,label
9389,While arguing over President Reagan’s 1981 tax...,Sarah Sanders,1
1861,"Recently Rick Scott ""closed 30 women’s health ...",Lois Frankel,0
11035,Says Target installed urinals in a women’s bat...,Facebook posts,0
12221,"Says ""combined doses of vaccines"" have never b...",Facebook posts,0
11354,: The AMBER Alert system has been discontinu...,NaN,0
...,...,...,...
2910,Health insurance costs for Floridians are up 3...,Republican Party of Florida,1
6096,"A photograph captures Harriet Tubman as a ""Gun...",NaN,0
10446,"ISIS leader Abu Bakr al Baghdadi was ""released...",Jeanine Pirro,0
5414,"""The board of a nonprofit organization on whic...",Tennessee Republican Party,1


In [9]:
unique_claimaints = claimaints.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)
unique_claimaints

,claim,claimant,label
9389,While arguing over President Reagan’s 1981 tax...,Sarah Sanders,1
1861,"Recently Rick Scott ""closed 30 women’s health ...",Lois Frankel,0
11035,Says Target installed urinals in a women’s bat...,Facebook posts,0
12221,"Says ""combined doses of vaccines"" have never b...",Facebook posts,0
8662,"Judge Gonzalo Curiel ""is giving us very unfair...",Donald Trump,0
...,...,...,...
5974,"""We have right now, … in the state of Californ...",Steve Cortes,1
11698,"""Transgender individuals in the U.S. have a 1-...",Garnet Coleman,0
2910,Health insurance costs for Floridians are up 3...,Republican Party of Florida,1
10446,"ISIS leader Abu Bakr al Baghdadi was ""released...",Jeanine Pirro,0


In [10]:
labelled_claimants = unique_claimaints.groupby('claimant')['label'].apply(list).to_dict()
labelled_claimants

{'"A Woman’s Right to Know Information Material”': [1],
 '"suburban mom" for Scott Taylor': [1],
 '@LagBeachAntifa9': [0],
 '@Sowellnomics': [1, 0],
 'A Facebook page': [0],
 'A Stronger Wisconsin': [1],
 'AARP': [0],
 'ABC NEWS-US': [0],
 'ACLU Foundation of Georgia': [2],
 'AFL-CIO': [1],
 'AFL-CIO of New Jersey': [1],
 'AFP Fact Check': [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
 'AFSCME': [1, 1, 0],
 'AFSCME People': [1, 1],
 'Aaron DeGroot': [1],
 'Abdikadir Mohamed': [0],
 'Abia Pulse News': [0],
 'Abigail Spanberger': [1],
 'Abubakar Bwari': [0],
 'ActionAid UK': [1],
 'Activist Mommy': [1],
 'Adam Hasner': [1, 1, 1, 2],
 'Adam Kinzinger': [0],
 'Adam Putnam': [1, 1, 0, 1],
 'Adam Schefter': [1],
 'Adam Schiff': [2, 0],
 'Addicting Information': [0],
 'Aden Duale': [0, 0],
 'Adrian Garcia': [1],
 'African National Congess': [1],
 'African National Congress': [2, 0, 0, 0],
 'Afrikan Daily': [0, 0, 0, 0],
 'Ahmednasir Abdullahi': [0, 0],
 'Ainsley Earhardt': [0],
 'Airline Amb

In [13]:
# from statistics import mode

# single_label_claimants = {}

# for claimant, label in labelled_claimants.items():
#     try:
#         single_label_claimants[claimant] = mode(label)
#     except:
#         single_label_claimants[claimant] = min(label)

# single_label_claimants

In [11]:
single_label_claimants = {}

for claimant, label in labelled_claimants.items():
    single_label_claimants[claimant] = round(sum(label) / len(label))

single_label_claimants

{'"A Woman’s Right to Know Information Material”': 1.0,
 '"suburban mom" for Scott Taylor': 1.0,
 '@LagBeachAntifa9': 0.0,
 '@Sowellnomics': 0.5,
 'A Facebook page': 0.0,
 'A Stronger Wisconsin': 1.0,
 'AARP': 0.0,
 'ABC NEWS-US': 0.0,
 'ACLU Foundation of Georgia': 2.0,
 'AFL-CIO': 1.0,
 'AFL-CIO of New Jersey': 1.0,
 'AFP Fact Check': 0.06666666666666667,
 'AFSCME': 0.6666666666666666,
 'AFSCME People': 1.0,
 'Aaron DeGroot': 1.0,
 'Abdikadir Mohamed': 0.0,
 'Abia Pulse News': 0.0,
 'Abigail Spanberger': 1.0,
 'Abubakar Bwari': 0.0,
 'ActionAid UK': 1.0,
 'Activist Mommy': 1.0,
 'Adam Hasner': 1.25,
 'Adam Kinzinger': 0.0,
 'Adam Putnam': 0.75,
 'Adam Schefter': 1.0,
 'Adam Schiff': 1.0,
 'Addicting Information': 0.0,
 'Aden Duale': 0.0,
 'Adrian Garcia': 1.0,
 'African National Congess': 1.0,
 'African National Congress': 0.5,
 'Afrikan Daily': 0.0,
 'Ahmednasir Abdullahi': 0.0,
 'Ainsley Earhardt': 0.0,
 'Airline Ambassadors International': 0.0,
 'Al Franken': 0.5,
 'Al Gore': 1.2,

In [12]:
single_label_claimants = {}

for claimant, label in labelled_claimants.items():
    single_label_claimants[claimant] = round(sum(label) / len(label))

single_label_claimants

{'"A Woman’s Right to Know Information Material”': 1,
 '"suburban mom" for Scott Taylor': 1,
 '@LagBeachAntifa9': 0,
 '@Sowellnomics': 0,
 'A Facebook page': 0,
 'A Stronger Wisconsin': 1,
 'AARP': 0,
 'ABC NEWS-US': 0,
 'ACLU Foundation of Georgia': 2,
 'AFL-CIO': 1,
 'AFL-CIO of New Jersey': 1,
 'AFP Fact Check': 0,
 'AFSCME': 1,
 'AFSCME People': 1,
 'Aaron DeGroot': 1,
 'Abdikadir Mohamed': 0,
 'Abia Pulse News': 0,
 'Abigail Spanberger': 1,
 'Abubakar Bwari': 0,
 'ActionAid UK': 1,
 'Activist Mommy': 1,
 'Adam Hasner': 1,
 'Adam Kinzinger': 0,
 'Adam Putnam': 1,
 'Adam Schefter': 1,
 'Adam Schiff': 1,
 'Addicting Information': 0,
 'Aden Duale': 0,
 'Adrian Garcia': 1,
 'African National Congess': 1,
 'African National Congress': 0,
 'Afrikan Daily': 0,
 'Ahmednasir Abdullahi': 0,
 'Ainsley Earhardt': 0,
 'Airline Ambassadors International': 0,
 'Al Franken': 0,
 'Al Gore': 1,
 'Al Jazeera': 0,
 'Al Jazeera America': 0,
 'Alan  Essig': 2,
 'Alan Grayson': 1,
 'Alberta Darling': 1,


In [14]:
pickle.dump( single_label_claimants, open( "../input/labelled_claimants2.p", "wb" ) )

## Top Claimaints

In [22]:
top_claimants = train_set['claimant'].value_counts()
top_claimants.columns = ['claimant', 'claims']
top_claimants_df = top_claimants.rename_axis('claimant').reset_index(name='claims')

In [24]:
top_claimants_df.head(10)

,claimant,claims
0,,3987
1,Donald Trump,987
2,Bloggers,304
3,Barack Obama,192
4,Hillary Clinton,172
5,Viral image,110
6,Ted Cruz,86
7,Facebook posts,81
8,Various websites,80
9,Marco Rubio,76


## False Claimants

In [25]:
false_claims = train_set.loc[train_set['label'] == 0]
false_claimants = false_claims['claimant'].value_counts()
false_claimants_df = false_claimants.rename_axis('claimant').reset_index(name='claims')

In [40]:
false_claimants_df.head(20)

,claimant,claims
0,,2441
1,Donald Trump,660
2,Bloggers,257
3,Viral image,101
4,Various websites,80
5,Facebook posts,61
6,multiple sources,58
7,Chain email,50
8,Facebook user,43
9,Hillary Clinton,41


In [27]:
top_false_claimants = false_claimants_df[false_claimants_df['claims'] > 0]

In [28]:
top_false_claimants

,claimant,claims
0,,2441
1,Donald Trump,660
2,Bloggers,257
3,Viral image,101
4,Various websites,80
...,...,...
1214,Tom Graves,1
1215,C.W. Bill Young,1
1216,Billy Richardson,1
1217,Rep. Xavier Becerra,1


In [29]:
labelled_false_claimants = pd.DataFrame(top_false_claimants, columns=['claimant'])
labelled_false_claimants['label'] = 0
labelled_false_claimants.drop(labelled_false_claimants.index[0], inplace=True) # Drop empty claimant row
labelled_false_claimants

,claimant,label
1,Donald Trump,0
2,Bloggers,0
3,Viral image,0
4,Various websites,0
5,Facebook posts,0
...,...,...
1214,Tom Graves,0
1215,C.W. Bill Young,0
1216,Billy Richardson,0
1217,Rep. Xavier Becerra,0


## Partly True Claimants

In [30]:
partly_claims = train_set.loc[train_set['label'] == 1]
partly_claimants = partly_claims['claimant'].value_counts()
partly_claimants_df = partly_claimants.rename_axis('claimant').reset_index(name='claims')

In [49]:
partly_claimants_df

,claimant,claims
0,,1026
1,Donald Trump,308
2,Barack Obama,119
3,Hillary Clinton,98
4,Ted Cruz,50
...,...,...
1639,Ohioans Against Deceptive Rx Ballot Issue,1
1640,Delaine Eastin,1
1641,Reclaim America PAC,1
1642,Angus King,1


In [50]:
is_viral =  partly_claims['claimant']=='Viral image'

In [51]:
viral = partly_claims[is_viral]
print(viral)

                                                   claim     claimant  \
2999   Says Denmark’s suicide rate has been about twi...  Viral image   
4960   Bernie Sanders opposes the Trans-Pacific Partn...  Viral image   
6343   "Tuition-free public universities and colleges...  Viral image   
15148  Chinese workers made "90,000 banners" for Trum...  Viral image   
3444   "After prohibition ended, police deaths didn't...  Viral image   
3643   Unlike Benghazi, there were no investigations ...  Viral image   
14658  "Illegal refugees get $3,874/mo. Under the Fed...  Viral image   
7181   Says after leaving the White House, Hillary Cl...  Viral image   
7631   "Snapchat CEO Evan Spiegel has said the app is...  Viral image   

             date  label                            related_articles     id  
2999   2015-10-22      1                              [27787, 83800]   3305  
4960   2015-08-26      1                       [79790, 79789, 79981]   5460  
6343   2016-06-01      1           

In [52]:
len(viral)

9

In [32]:
top_partly_claimants = partly_claimants_df[partly_claimants_df['claims'] > 0]

In [33]:
top_partly_claimants

,claimant,claims
0,,1026
1,Donald Trump,308
2,Barack Obama,119
3,Hillary Clinton,98
4,Ted Cruz,50
...,...,...
1639,Ohioans Against Deceptive Rx Ballot Issue,1
1640,Delaine Eastin,1
1641,Reclaim America PAC,1
1642,Angus King,1


In [34]:
labelled_partly_claimants = pd.DataFrame(top_partly_claimants, columns=['claimant'])
labelled_partly_claimants['label'] = 1
labelled_partly_claimants.drop(labelled_partly_claimants.index[0], inplace=True)
labelled_partly_claimants

,claimant,label
1,Donald Trump,1
2,Barack Obama,1
3,Hillary Clinton,1
4,Ted Cruz,1
5,Bernie Sanders,1
...,...,...
1639,Ohioans Against Deceptive Rx Ballot Issue,1
1640,Delaine Eastin,1
1641,Reclaim America PAC,1
1642,Angus King,1


## True Claimants

In [35]:
true_claims = train_set.loc[train_set['label'] == 2]
true_claimants = true_claims['claimant'].value_counts()
true_claimants_df = true_claimants.rename_axis('claimant').reset_index(name='claims')

In [43]:
true_claimants_df.head(50)

,claimant,claims
0,,520
1,Barack Obama,38
2,Hillary Clinton,33
3,Donald Trump,19
4,John McCain,14
5,President Cyril Ramaphosa,13
6,Scott Walker,11
7,Rick Perry,10
8,Jeb Bush,10
9,Bernie Sanders,9


In [37]:
top_true_claimants = true_claimants_df[true_claimants_df['claims'] > 0]

In [38]:
top_true_claimants

,claimant,claims
0,,520
1,Barack Obama,38
2,Hillary Clinton,33
3,Donald Trump,19
4,John McCain,14
...,...,...
519,Habitat for Humanity of Broward,1
520,Mmusi Maimane,1
521,Scott Armstrong,1
522,The South African,1


In [39]:
labelled_true_claimants = pd.DataFrame(top_true_claimants, columns=['claimant'])
labelled_true_claimants['label'] = 2
labelled_true_claimants.drop(labelled_true_claimants.index[0], inplace=True)
labelled_true_claimants

,claimant,label
1,Barack Obama,2
2,Hillary Clinton,2
3,Donald Trump,2
4,John McCain,2
5,President Cyril Ramaphosa,2
...,...,...
519,Habitat for Humanity of Broward,2
520,Mmusi Maimane,2
521,Scott Armstrong,2
522,The South African,2


In [53]:
is_viral =  true_claims['claimant']=='Viral image'
viral = true_claims[is_viral]
print(viral)

Empty DataFrame
Columns: [claim, claimant, date, label, related_articles, id]
Index: []
